In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pdb
import os
import math
import pysam
import sys
import lockfile
from time import sleep

sys.path.append('/home/rt2776/cnv_espresso/src')
import function as func
import function_dl as func_dl

Tensorflow version 2.2.0


In [31]:

def generate_one_image(cnv_data_df, sge_task_id, col_dict, 
                        RD_norm_dir, ref_samples_dir, output_path, corr_threshold=-1, flanking=None):
    index = sge_task_id-1
    row   = cnv_data_df.iloc[index]
    col_sampleID       = col_dict['col_sampleID']
    col_cnv_interval   = col_dict['col_cnv_interval']
    col_cnv_chr        = col_dict['col_cnv_chr']
    col_cnv_start      = col_dict['col_cnv_start']
    col_cnv_end        = col_dict['col_cnv_end']
    col_cnv_type       = col_dict['col_cnv_type']
    col_cnv_num_targets= col_dict['col_cnv_num_targets']
    col_cnv_canoes     = col_dict['col_cnv_canoes']
    col_cnv_xhmm       = col_dict['col_cnv_xhmm']
    col_cnv_clamms     = col_dict['col_cnv_clamms']
    col_cnv_numCarriers= col_dict['col_cnv_numCarriers']
    col_GSD_label      = col_dict['col_GSD_label']
    col_PRE_label      = col_dict['col_PRE_label']

    output_EntireCNV_image_dir  = output_path 

    sampleID  = row[col_sampleID]
    try:
        cnv_interval = row[col_cnv_interval]
        cnv_chr, cnv_start, cnv_end = func.parseInterval(cnv_interval) 
    except:
        cnv_chr   = row[col_cnv_chr]
        cnv_start = int(row[col_cnv_start])
        cnv_end   = int(row[col_cnv_end])
    cnv_type  = row[col_cnv_type]

    if cnv_type == 1:
        cnv_type = "DEL"
    elif cnv_type == 3:
        cnv_type = "DUP"
    else:
        pass

    case_sample_color = color_del if cnv_type == 'DEL' else color_dup

    cnv_num_targets   = row[col_cnv_num_targets] if col_cnv_num_targets != None else 'NA'
    cnv_canoes        = str(row[col_cnv_canoes]) if col_cnv_canoes != None else 'NA'
    cnv_xhmm          = str(row[col_cnv_xhmm])   if col_cnv_xhmm   != None else 'NA'
    cnv_clamms        = str(row[col_cnv_clamms]) if col_cnv_clamms != None else 'NA'
    cnv_num_carriers  = str(row[col_cnv_numCarriers]) if col_cnv_numCarriers != None else 'NA'
    cnv_gsd_label     = row[col_GSD_label] if col_GSD_label != None else 'NA'
    cnv_CNLearn_label = row[col_PRE_label] if col_PRE_label != None else 'NA'

    if cnv_gsd_label == 'NA':
        cnv_gsd_str = 'NA'
    else:    
        if cnv_gsd_label == 1:
            cnv_gsd_str = "True"
        elif cnv_gsd_label == 0:
            cnv_gsd_str = "False"
        else:
            cnv_gsd_str = cnv_gsd_label 

    if cnv_CNLearn_label == 'NA':
        cnv_CNLearn_str = 'NA'
    else:    
        cnv_CNLearn_str = "True" if cnv_CNLearn_label == 1 else "False"

    print("[%d|%d] Illustrating: %s %s:%d-%d %s #targets:%s Label:%s"% \
           (len(cnv_data_df), index+1, sampleID, cnv_chr, cnv_start, cnv_end, cnv_type, str(cnv_num_targets), cnv_gsd_str))

    ## Confirm the boundries
    if flanking == True or flanking == 'True':
        cnv_length   = cnv_end - cnv_start + 1
        figure_left  = cnv_start - cnv_length/2
        figure_right = cnv_end + cnv_length/2
    else:
        figure_left  = cnv_start
        figure_right = cnv_end

    print(" Generate CNV %s:%d-%d %s with boundary located at %s:%d-%d"%(cnv_chr, cnv_start, cnv_end, cnv_type, 
                                                                        cnv_chr, figure_left, figure_right))
    ## Import RD data info
    print("  --Step1. Fetching RD data for case sample ...")
    RD_cnv_region_df = func.fetchRDdata_byTabix(RD_norm_dir, sampleID, cnv_chr, figure_left, figure_right, fixed_win_num)

    ## Fetch Read depth data for reference samples in terms of CNV boundary       
    print("  --Step2. Fetching RD data for reference samples ...")
    ref_samples_file = func.fetch_relative_file_path(ref_samples_dir, sampleID,'txt')

    if not os.path.exists(ref_samples_file):
        print("    -[Error]: error in reference samples related file for %s in %s"%(sampleID, ref_samples_dir))
        exit(0)
  
    reference_RD_df = func.fetchRefRDdata_byTabix(RD_norm_dir, ref_samples_file, 
                                                    cnv_chr, figure_left, figure_right, 
                                                    fixed_win_num, corr_threshold)

    ## plot the entire cnv into one image
    print("  --Step3. Illustrating an image for the entire CNV ...")
    title_info = sampleID+" "+str(cnv_chr)+":"+str(cnv_start)+"-"+str(cnv_end)+" "+cnv_type + \
                 " "+ str((cnv_end-cnv_start)/1000) + 'kb'+ " #targets:" + str(len(RD_cnv_region_df)) 

    image_file = str(index+1)+"_"+sampleID+"_"+str(cnv_chr)+"_"+str(cnv_start)+"_"+str(cnv_end) + \
                 "_"+str(cnv_num_targets)+"tgs_"+str(len(RD_cnv_region_df))+"wins_"+cnv_type+".pdf"

    fig = plt.figure(dpi=150,figsize=(10, 7)) 
    ax_rd = fig.subplots(nrows=1, ncols=1)

    ### plot reference samples
    for sample_reader in reference_RD_df["sample"].unique():
                ref_sample_df = reference_RD_df[reference_RD_df["sample"]==sample_reader]
                ax_rd.plot((ref_sample_df["start"]+ref_sample_df["end"])/2, ref_sample_df["RD_norm"],
                            color='grey', marker='.', linewidth=0.2)

    ### plot case sample
    ax_rd.plot((RD_cnv_region_df["start"]+RD_cnv_region_df["end"])/2, RD_cnv_region_df["RD_norm"], \
                color=case_sample_color , marker='o', linewidth=2)
    ax_rd.set_title(title_info)

    ### write the img path to the cnv_file_w_img_file
    print("  --Step4. Output image file to %s."%(output_EntireCNV_image_dir+image_file))
    img_path = output_EntireCNV_image_dir+image_file
    plt.savefig(img_path)
    plt.close()  
    print("  --[Done].")
    

In [10]:
RD_norm_dir     = '/home/rt2776/cnv_espresso/project1_pcgc/norm/'
ref_samples_dir = '/home/rt2776/cnv_espresso/project1_pcgc/ref_samples/'
cnv_file        = '/home/rt2776/cnv_espresso/project1_pcgc/images_new/pcgc_NimbleGenV2_data_withImagePath.csv'
output_path     = '/home/rt2776/cnv_espresso/project1_pcgc/'
cnv_data_df = pd.read_csv(cnv_file)
cnv_data_df

,ID,Chr,Start,End,Band,CNV_TYPE,Syndrome_or_Gene,Technologies,CardiacLesion(Diagnosis),ParentOrigin,Extracardiac,Genes_n,Size_kb,ref,batch,Num_Targets_Wins,entire_cnv_path,split_cnv_path
0,1-01401,1,59247993,59251097,p32.1,1,JUN,A,LVOT(HLHS),-,-,1,3.1,hg19,NimbleGenV2,1,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
1,1-03171,1,145586403,145799634,q21.1,3,"1q21.1,dup_or_GJA5‡","A,E",CTD(TOF_or_APVS),-,-,7,213.2,hg19,NimbleGenV2,50,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
2,1-01036,1,146631133,147416212,q21.1,3,"1q21.1,dup_or_GJA5‡",E,CTD(TOF),M,-,15,785.1,hg19,NimbleGenV2,75,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
3,1-01486,1,194201171,194304070,q24.2–q25,3,CDC73,A,LVOT(HLHS),-,Yes,0,102.9,hg19,NimbleGenV2,0,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
4,1-01536,2,70168995,70359345,p13.3,1,PCBP1,A,CTD(TOF_or_PA),-,-,5,190.4,hg19,NimbleGenV2,4,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
5,1-01401,2,102493466,103001458,q11.2–q12.1,1,MAP4K4,E,LVOT(HLHS),-,-,6,508.0,hg19,NimbleGenV2,52,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
6,1-01401,2,145155868,145274931,q22.3,1,Mowat-Wilson_or_ZEB2‡,E,LVOT(HLHS),-,-,1,119.1,hg19,NimbleGenV2,9,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
7,1-00762,3,60661,11712230,p26.1,3,"ARL8B,ARPC4,CAMK1,CAV3,CRBN,EMC3,ITPR1,SEC13,S...",A,ASD_or_PS(ASD),-,Yes,103,11651.6,hg19,NimbleGenV2,664,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
8,1-01049,3,15637812,15643461,p25.1,3,"BTD,HACL1",E,CTD(TOF),-,-,2,5.6,hg19,NimbleGenV2,4,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
9,1-01045,3,47780965,48309270,p21.31,3,"CDC25A,DHX30,MAP4,SMARCC1",A,LVOT(HLHS),-,-,14,528.3,hg19,NimbleGenV2,69,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN


In [15]:
## Output images and folders
output_EntireCNV_image_dir  = output_path 
os.makedirs(output_EntireCNV_image_dir, exist_ok=True)

## Variables
global_var_dict = func.global_variables()
SAMPLE          = global_var_dict['SAMPLE']
CNV_CHR         = global_var_dict['CNV_CHR']
CNV_START       = global_var_dict['CNV_START']
CNV_END         = global_var_dict['CNV_END']
CNV_INTERVAL    = global_var_dict['CNV_INTERVAL']
CNV_TYPE        = global_var_dict['CNV_TYPE']
NUM_TARGETS     = global_var_dict['NUM_TARGETS']
GSD_LABEL       = global_var_dict['CNV_LABEL']
PRE_LABEL       = global_var_dict['REF']

fixed_win_num = 3 # the number of targets per group
color_del, color_dup = (0,0,1), (0,0,1) #blue for three classes labels

## Parse header
cnv_data_header  = cnv_data_df.columns.tolist()
col_sampleID     = func.fetch_colName(SAMPLE, cnv_data_header)[0]
col_cnv_interval = func.fetch_colName(CNV_INTERVAL, cnv_data_header)[0]
col_cnv_chr      = func.fetch_colName(CNV_CHR, cnv_data_header)[0]
col_cnv_start    = func.fetch_colName(CNV_START, cnv_data_header)[0]
col_cnv_end      = func.fetch_colName(CNV_END, cnv_data_header)[0]
col_cnv_type     = func.fetch_colName(CNV_TYPE, cnv_data_header)[0]
col_GSD_label    = func.fetch_colName(GSD_LABEL, cnv_data_header)[0]
col_PRE_label    = func.fetch_colName(PRE_LABEL, cnv_data_header)[0]
col_cnv_canoes   = func.fetch_colName(['CANOES','CANOES_RT'], cnv_data_header)[0]
col_cnv_xhmm     = func.fetch_colName(['XHMM','XHMM_RT'], cnv_data_header)[0]
col_cnv_clamms   = func.fetch_colName(['CLAMMS','CLAMMS_RT'], cnv_data_header)[0]
col_cnv_num_targets = func.fetch_colName(NUM_TARGETS, cnv_data_header)[0]
col_cnv_numCarriers = func.fetch_colName(['Num_Carriers(inGivenCohort)'], cnv_data_header)[0]

col_dict = {
    'col_sampleID'     : col_sampleID,
    'col_cnv_interval' : col_cnv_interval,
    'col_cnv_chr'      : col_cnv_chr,
    'col_cnv_start'    : col_cnv_start,
    'col_cnv_end'      : col_cnv_end,
    'col_cnv_type'     : col_cnv_type,
    'col_cnv_num_targets': col_cnv_num_targets,
    'col_cnv_canoes'   : col_cnv_canoes,
    'col_cnv_xhmm'     : col_cnv_xhmm,
    'col_cnv_clamms'   : col_cnv_clamms,
    'col_cnv_numCarriers': col_cnv_numCarriers,
    'col_GSD_label'    : col_GSD_label,
    'col_PRE_label'    : col_PRE_label
}


In [33]:
cnv_data_df

,ID,Chr,Start,End,Band,CNV_TYPE,Syndrome_or_Gene,Technologies,CardiacLesion(Diagnosis),ParentOrigin,Extracardiac,Genes_n,Size_kb,ref,batch,Num_Targets_Wins,entire_cnv_path,split_cnv_path
0,1-01401,1,59247993,59251097,p32.1,1,JUN,A,LVOT(HLHS),-,-,1,3.1,hg19,NimbleGenV2,1,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
1,1-03171,1,145586403,145799634,q21.1,3,"1q21.1,dup_or_GJA5‡","A,E",CTD(TOF_or_APVS),-,-,7,213.2,hg19,NimbleGenV2,50,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
2,1-01036,1,146631133,147416212,q21.1,3,"1q21.1,dup_or_GJA5‡",E,CTD(TOF),M,-,15,785.1,hg19,NimbleGenV2,75,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
3,1-01486,1,194201171,194304070,q24.2–q25,3,CDC73,A,LVOT(HLHS),-,Yes,0,102.9,hg19,NimbleGenV2,0,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
4,1-01536,2,70168995,70359345,p13.3,1,PCBP1,A,CTD(TOF_or_PA),-,-,5,190.4,hg19,NimbleGenV2,4,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
5,1-01401,2,102493466,103001458,q11.2–q12.1,1,MAP4K4,E,LVOT(HLHS),-,-,6,508.0,hg19,NimbleGenV2,52,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
6,1-01401,2,145155868,145274931,q22.3,1,Mowat-Wilson_or_ZEB2‡,E,LVOT(HLHS),-,-,1,119.1,hg19,NimbleGenV2,9,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
7,1-00762,3,60661,11712230,p26.1,3,"ARL8B,ARPC4,CAMK1,CAV3,CRBN,EMC3,ITPR1,SEC13,S...",A,ASD_or_PS(ASD),-,Yes,103,11651.6,hg19,NimbleGenV2,664,/home/rt2776/cnv_espresso/project1_pcgc/images...,/home/rt2776/cnv_espresso/project1_pcgc/images...
8,1-01049,3,15637812,15643461,p25.1,3,"BTD,HACL1",E,CTD(TOF),-,-,2,5.6,hg19,NimbleGenV2,4,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN
9,1-01045,3,47780965,48309270,p21.31,3,"CDC25A,DHX30,MAP4,SMARCC1",A,LVOT(HLHS),-,-,14,528.3,hg19,NimbleGenV2,69,/home/rt2776/cnv_espresso/project1_pcgc/images...,NaN


In [23]:
print(RD_norm_dir)
#print(cnv_info_w_img_file)
print(ref_samples_dir)
print(output_path)

/home/rt2776/cnv_espresso/project1_pcgc/norm/
/home/rt2776/cnv_espresso/project1_pcgc/ref_samples/
/home/rt2776/cnv_espresso/project1_pcgc/


In [32]:
# Figure 2a
generate_one_image(cnv_data_df, 39, col_dict, RD_norm_dir, ref_samples_dir, output_path)

[52|39] Illustrating: 1-00561 17:27962393-28099002 DEL #targets:NA Label:NA
 Generate CNV 17:27962393-28099002 DEL with boundary located at 17:27962393-28099002
  --Step1. Fetching RD data for case sample ...
  --Step2. Fetching RD data for reference samples ...
  --Step3. Illustrating an image for the entire CNV ...
  --Step4. Output image file to /home/rt2776/cnv_espresso/project1_pcgc/39_1-00561_17_27962393_28099002_NAtgs_12wins_DEL.pdf.
  --[Done].


In [34]:
# Figure 2b
generate_one_image(cnv_data_df, 50, col_dict, RD_norm_dir, ref_samples_dir, output_path)

[52|50] Illustrating: 1-01427 22:42522638-42531210 DUP #targets:NA Label:NA
 Generate CNV 22:42522638-42531210 DUP with boundary located at 22:42522638-42531210
  --Step1. Fetching RD data for case sample ...
  --Step2. Fetching RD data for reference samples ...
  --Step3. Illustrating an image for the entire CNV ...
  --Step4. Output image file to /home/rt2776/cnv_espresso/project1_pcgc/50_1-01427_22_42522638_42531210_NAtgs_9wins_DUP.pdf.
  --[Done].
